# Research: Diagnostic et Correction de la Stratégie ETF Pairs Trading

## Contexte

La stratégie actuelle affiche un **Sharpe ratio de -0.759** sur la période 2020-2024, ce qui signifie qu'elle **perd de l'argent**. Cette recherche vise à identifier les causes profondes et proposer des corrections basées sur des données empiriques.

## Problématique

Le pairs trading repose sur l'hypothèse de cointégration : deux actifs évoluent ensemble à long terme, et leurs écarts temporaires (spreads) reviennent vers une moyenne. Pourquoi cette stratégie échoue-t-elle ?

### Pièges courants du pairs trading

1. **Divergence du spread** : Les paires perdent leur cointégration en cours de position
2. **Sortie prématurée** : Exit à z-score = 0.5 au lieu de 0.0 (mean)
3. **Instabilité du beta** : EWMA peut amplifier le bruit à court terme
4. **Coûts de transaction** : Sur-trading avec des half-lives trop courts
5. **Mauvaise gestion du risque** : Stop-loss par jambe au lieu de spread-level
6. **Durée d'insight inadaptée** : 6h fixes au lieu de se baser sur le half-life

## Hypothèses à tester

1. **H1** : Corriger z-exit de 0.5 → 0.0 améliore le Sharpe de >0.3 points
2. **H2** : Les paires restent cointégrées <50% du temps après 1 semaine
3. **H3** : Half-life moyen > 10 jours (trop long pour du hourly trading)
4. **H4** : Un filtre half-life < 30 jours élimine les paires instables
5. **H5** : Walk-forward validation montre une dégradation >30% hors-sample

## Méthodologie

1. Charger les données ETF sectoriels (2015-2026) avec QuantBook
2. Analyser la cointégration et stabilité temporelle des paires
3. Calculer les half-lives de mean-reversion
4. Backtester vectorisé avec différents paramètres (z-exit, stop-loss)
5. Walk-forward validation (train 1 an, trade 3 mois)
6. Synthétiser les corrections prioritaires

In [ ]:
# Cell 2: Setup QuantBook et chargement des données
from AlgorithmImports import *
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from itertools import combinations
import warnings
warnings.filterwarnings('ignore')

# Initialiser QuantBook
qb = QuantBook()

# Secteur ETFs du S&P (SPDR)
etf_tickers = ["XLB", "XLE", "XLF", "XLI", "XLK", "XLP", "XLU", "XLV", "XLY"]
symbols = {}

for ticker in etf_tickers:
    symbols[ticker] = qb.AddEquity(ticker, Resolution.Daily).Symbol

# Charger les données historiques (2015-01-01 → maintenant)
start_date = datetime(2015, 1, 1)
end_date = datetime.now()

history = qb.History(list(symbols.values()), start_date, end_date, Resolution.Daily)

# Extraire les prix de clôture
if 'close' in history.columns:
    prices = history['close'].unstack(level=0)
    # Renommer les colonnes avec les tickers
    prices.columns = [s.Value for s in prices.columns]
    prices = prices[etf_tickers]  # Réordonner
    prices = prices.dropna()
    
    print(f"✓ Chargé {len(prices)} jours de données pour {len(prices.columns)} ETFs")
    print(f"  Période: {prices.index[0].date()} → {prices.index[-1].date()}")
    print(f"\nAperçu:")
    print(prices.head())
else:
    print("⚠ Erreur: colonne 'close' non trouvée dans l'historique")
    prices = pd.DataFrame()

In [ ]:
# Cell 3: Analyse de cointégration - test de toutes les paires
from statsmodels.tsa.stattools import coint
import scipy.stats as stats

def test_cointegration(p1, p2, pvalue_threshold=0.05):
    """
    Test de cointégration d'Engle-Granger.
    
    Returns:
        dict: {'cointegrated': bool, 'pvalue': float, 'statistic': float}
    """
    try:
        score, pvalue, _ = coint(p1, p2)
        return {
            'cointegrated': pvalue < pvalue_threshold,
            'pvalue': round(pvalue, 4),
            'statistic': round(score, 4)
        }
    except:
        return {'cointegrated': False, 'pvalue': 1.0, 'statistic': 0.0}

# Générer toutes les paires possibles
all_pairs = list(combinations(etf_tickers, 2))
print(f"Analyse de {len(all_pairs)} paires possibles\n")

# Tester chaque paire sur la période complète
coint_results = []

for pair in all_pairs:
    etf1, etf2 = pair
    p1 = prices[etf1]
    p2 = prices[etf2]
    
    result = test_cointegration(p1, p2)
    coint_results.append({
        'pair': f"{etf1}-{etf2}",
        'etf1': etf1,
        'etf2': etf2,
        **result
    })

coint_df = pd.DataFrame(coint_results).sort_values('pvalue')

# Statistiques globales
n_cointegrated = coint_df['cointegrated'].sum()
print(f"Paires cointégrées (p < 0.05): {n_cointegrated}/{len(all_pairs)} ({100*n_cointegrated/len(all_pairs):.1f}%)\n")

# Top 10 paires par p-value
print("Top 10 paires les plus cointégrées:")
print(coint_df[['pair', 'pvalue', 'statistic']].head(10).to_string(index=False))

# Bottom 5 (les moins cointégrées)
print("\nPires 5 paires (moins cointégrées):")
print(coint_df[['pair', 'pvalue', 'statistic']].tail(5).to_string(index=False))

In [ ]:
# Cell 4: Stabilité temporelle des paires - rolling cointegration test
def rolling_cointegration(p1, p2, window_days=252, step_days=63):
    """
    Test de cointégration glissant pour mesurer la stabilité temporelle.
    
    Args:
        window_days: Fenêtre de test (252 jours = 1 an)
        step_days: Pas de roulement (63 jours = ~3 mois)
    
    Returns:
        list: Historique des p-values
    """
    pvalues = []
    dates = []
    
    for i in range(window_days, len(p1), step_days):
        window_p1 = p1.iloc[i-window_days:i]
        window_p2 = p2.iloc[i-window_days:i]
        
        result = test_cointegration(window_p1, window_p2)
        pvalues.append(result['pvalue'])
        dates.append(p1.index[i])
    
    return pd.Series(pvalues, index=dates)

# Tester la stabilité des 5 meilleures paires
print("Analyse de stabilité temporelle (rolling 1-year cointegration)\n")

stability_results = {}

for _, row in coint_df.head(5).iterrows():
    pair_name = row['pair']
    etf1, etf2 = row['etf1'], row['etf2']
    
    pvalue_history = rolling_cointegration(prices[etf1], prices[etf2])
    stability_results[pair_name] = pvalue_history
    
    # Calculer le % de temps où la paire est cointégrée
    stable_pct = (pvalue_history < 0.05).sum() / len(pvalue_history) * 100
    
    print(f"{pair_name}:")
    print(f"  P-value moyenne: {pvalue_history.mean():.4f}")
    print(f"  Stable (p<0.05): {stable_pct:.1f}% du temps")
    print(f"  Périodes testées: {len(pvalue_history)}")
    print()

# Conclusion sur H2
avg_stability = np.mean([((pv < 0.05).sum() / len(pv) * 100) for pv in stability_results.values()])
print(f"⚠ Stabilité moyenne des top 5 paires: {avg_stability:.1f}%")
print(f"   {'✓ H2 CONFIRMÉE' if avg_stability < 50 else '✗ H2 INFIRMÉE'}: Les paires restent cointégrées {'<' if avg_stability < 50 else '>'}50% du temps")

In [ ]:
# Cell 5: Calcul du half-life de mean-reversion
import statsmodels.api as sm

def compute_spread_and_half_life(p1, p2):
    """
    Calcule le spread cointégré et son half-life de mean-reversion.
    
    Méthode: OLS regression du spread_lag sur spread_diff
    Half-life = -ln(2) / beta si beta < 0
    
    Returns:
        dict: {'spread': Series, 'beta': float, 'half_life_days': float}
    """
    # 1. Estimer le beta de la paire (OLS)
    X = sm.add_constant(p2)
    model = sm.OLS(p1, X)
    result = model.fit()
    beta = result.params[1]
    
    # 2. Calculer le spread
    spread = p1 - beta * p2
    
    # 3. Estimer le half-life (AR(1) mean-reversion speed)
    spread_lag = spread.shift(1).dropna()
    spread_diff = spread.diff().dropna()
    
    # Aligner les indices
    common_idx = spread_lag.index.intersection(spread_diff.index)
    X_ar = sm.add_constant(spread_lag.loc[common_idx])
    y_ar = spread_diff.loc[common_idx]
    
    model_ar = sm.OLS(y_ar, X_ar)
    result_ar = model_ar.fit()
    
    # Beta AR(1) (coefficient du spread_lag)
    beta_ar = result_ar.params[1]
    
    if beta_ar < 0:
        half_life = -np.log(2) / beta_ar
    else:
        # Pas de mean-reversion si beta_ar >= 0
        half_life = float('inf')
    
    return {
        'spread': spread,
        'beta': round(beta, 4),
        'beta_ar': round(beta_ar, 4),
        'half_life_days': round(half_life, 2)
    }

# Calculer le half-life pour toutes les paires cointégrées
print("Calcul des half-lives de mean-reversion\n")

half_life_results = []

for _, row in coint_df[coint_df['cointegrated']].iterrows():
    pair_name = row['pair']
    etf1, etf2 = row['etf1'], row['etf2']
    
    result = compute_spread_and_half_life(prices[etf1], prices[etf2])
    
    half_life_results.append({
        'pair': pair_name,
        'pvalue': row['pvalue'],
        'beta': result['beta'],
        'half_life_days': result['half_life_days']
    })

hl_df = pd.DataFrame(half_life_results).sort_values('half_life_days')

# Statistiques globales
valid_hl = hl_df[hl_df['half_life_days'] < 365]  # Exclure les inf
print(f"Paires avec half-life valide (<365j): {len(valid_hl)}/{len(hl_df)}\n")

if len(valid_hl) > 0:
    print(f"Half-life moyen: {valid_hl['half_life_days'].mean():.1f} jours")
    print(f"Half-life médian: {valid_hl['half_life_days'].median():.1f} jours")
    print(f"Half-life min: {valid_hl['half_life_days'].min():.1f} jours")
    print(f"Half-life max: {valid_hl['half_life_days'].max():.1f} jours\n")
    
    # Filtrage par half-life
    fast_pairs = valid_hl[valid_hl['half_life_days'] < 30]
    print(f"Paires avec HL < 30 jours: {len(fast_pairs)} ({100*len(fast_pairs)/len(valid_hl):.1f}%)")
    
    # Top 10 paires par half-life court
    print("\nTop 10 paires avec mean-reversion rapide:")
    print(hl_df[['pair', 'pvalue', 'half_life_days']].head(10).to_string(index=False))
    
    # Conclusion sur H3
    median_hl = valid_hl['half_life_days'].median()
    print(f"\n{'✓ H3 CONFIRMÉE' if median_hl > 10 else '✗ H3 INFIRMÉE'}: Half-life médian = {median_hl:.1f} jours {'>' if median_hl > 10 else '<='} 10 jours")
else:
    print("⚠ Aucune paire avec half-life valide trouvée")

In [ ]:
# Cell 6: Backtest vectorisé avec différents z-score exit
def pairs_trading_backtest(p1, p2, z_entry=1.5, z_exit=0.0, lookback_ols=252, lookback_zscore=60):
    """
    Backtest vectorisé d'une stratégie pairs trading.
    
    Args:
        z_entry: Seuil d'entrée (ex: 1.5)
        z_exit: Seuil de sortie (ex: 0.0 pour mean)
        lookback_ols: Fenêtre de calcul du beta
        lookback_zscore: Fenêtre de calcul du z-score
    
    Returns:
        dict: Métriques de performance
    """
    # Calculer le beta rolling (OLS)
    betas = []
    spreads = []
    
    for i in range(lookback_ols, len(p1)):
        window_p1 = p1.iloc[i-lookback_ols:i]
        window_p2 = p2.iloc[i-lookback_ols:i]
        
        # OLS beta
        beta = np.cov(window_p1, window_p2)[0,1] / np.var(window_p2)
        spread = p1.iloc[i] - beta * p2.iloc[i]
        
        betas.append(beta)
        spreads.append(spread)
    
    spread_series = pd.Series(spreads, index=p1.index[lookback_ols:])
    
    # Calculer le z-score du spread
    spread_mean = spread_series.rolling(lookback_zscore).mean()
    spread_std = spread_series.rolling(lookback_zscore).std()
    z_score = (spread_series - spread_mean) / spread_std
    z_score = z_score.dropna()
    
    # Générer les signaux de trading
    signal = pd.Series(0.0, index=z_score.index)
    position = 0
    entries = 0
    exits = 0
    
    for i in range(1, len(z_score)):
        z = z_score.iloc[i]
        
        if position == 0:
            # Entrée long spread (long etf1, short etf2)
            if z < -z_entry:
                position = 1
                entries += 1
            # Entrée short spread (short etf1, long etf2)
            elif z > z_entry:
                position = -1
                entries += 1
        else:
            # Sortie long spread
            if position == 1 and z > z_exit:
                position = 0
                exits += 1
            # Sortie short spread
            elif position == -1 and z < -z_exit:
                position = 0
                exits += 1
        
        signal.iloc[i] = position
    
    # Calculer les returns du spread
    spread_returns = spread_series.pct_change()
    spread_returns = spread_returns.loc[signal.index]
    
    # Returns de la stratégie (signal décalé d'1 jour)
    strategy_returns = spread_returns * signal.shift(1)
    strategy_returns = strategy_returns.dropna()
    
    # Métriques
    if len(strategy_returns) > 0 and strategy_returns.std() > 0:
        sharpe = strategy_returns.mean() / strategy_returns.std() * np.sqrt(252)
        cumulative_returns = (1 + strategy_returns).cumprod()
        max_dd = (cumulative_returns / cumulative_returns.cummax() - 1).min()
        win_rate = (strategy_returns > 0).sum() / len(strategy_returns) if len(strategy_returns) > 0 else 0
    else:
        sharpe = 0
        max_dd = 0
        win_rate = 0
    
    return {
        'sharpe': round(sharpe, 3),
        'max_dd': round(max_dd * 100, 2),
        'n_trades': entries,
        'win_rate': round(win_rate * 100, 1)
    }

# Tester différentes valeurs de z_exit sur les 3 meilleures paires
print("Backtest avec différents seuils de z-score exit\n")

z_exit_values = [0.5, 0.25, 0.0, -0.25]  # Négatif = attendre l'overshoot
test_pairs = hl_df.head(3)

results_by_zexit = []

for z_exit in z_exit_values:
    print(f"\n=== Z-exit = {z_exit} ===")
    
    for _, row in test_pairs.iterrows():
        pair = row['pair']
        etf1, etf2 = pair.split('-')
        
        metrics = pairs_trading_backtest(
            prices[etf1], prices[etf2],
            z_entry=1.5,
            z_exit=z_exit
        )
        
        results_by_zexit.append({
            'z_exit': z_exit,
            'pair': pair,
            **metrics
        })
        
        print(f"{pair}: Sharpe={metrics['sharpe']}, MaxDD={metrics['max_dd']}%, Trades={metrics['n_trades']}, WinRate={metrics['win_rate']}%")

# Analyser l'impact du z_exit
results_df = pd.DataFrame(results_by_zexit)
avg_by_zexit = results_df.groupby('z_exit')['sharpe'].mean()

print("\n=== Impact moyen du z-exit ===")
for z, sharpe in avg_by_zexit.items():
    print(f"Z-exit = {z:5.2f} → Sharpe moyen = {sharpe:6.3f}")

# Conclusion sur H1
sharpe_baseline = avg_by_zexit.get(0.5, 0)
sharpe_corrected = avg_by_zexit.get(0.0, 0)
improvement = sharpe_corrected - sharpe_baseline

print(f"\n{'✓ H1 CONFIRMÉE' if improvement > 0.3 else '✗ H1 INFIRMÉE'}: Amélioration Sharpe (0.5→0.0) = {improvement:+.3f} points")

In [ ]:
# Cell 7: Comparaison des approches de stop-loss
def pairs_trading_with_stop(p1, p2, stop_type='spread_sigma', stop_threshold=2.5):
    """
    Backtest avec différentes stratégies de stop-loss.
    
    Args:
        stop_type: 'per_leg' (8% par jambe), 'spread_sigma' (2.5σ du spread), 'time_based'
        stop_threshold: Valeur du seuil selon le type
    """
    lookback_ols = 252
    lookback_zscore = 60
    z_entry = 1.5
    z_exit = 0.0
    
    # Calculer spread et z-score (même logique que précédemment)
    betas = []
    spreads = []
    
    for i in range(lookback_ols, len(p1)):
        window_p1 = p1.iloc[i-lookback_ols:i]
        window_p2 = p2.iloc[i-lookback_ols:i]
        beta = np.cov(window_p1, window_p2)[0,1] / np.var(window_p2)
        spread = p1.iloc[i] - beta * p2.iloc[i]
        spreads.append(spread)
    
    spread_series = pd.Series(spreads, index=p1.index[lookback_ols:])
    spread_mean = spread_series.rolling(lookback_zscore).mean()
    spread_std = spread_series.rolling(lookback_zscore).std()
    z_score = (spread_series - spread_mean) / spread_std
    z_score = z_score.dropna()
    
    # Générer les signaux avec stop-loss
    signal = pd.Series(0.0, index=z_score.index)
    position = 0
    entry_spread = None
    entry_date = None
    stopped_out = 0
    
    for i in range(1, len(z_score)):
        z = z_score.iloc[i]
        current_spread = spread_series.loc[z_score.index[i]]
        
        if position == 0:
            # Entrée
            if z < -z_entry:
                position = 1
                entry_spread = current_spread
                entry_date = z_score.index[i]
            elif z > z_entry:
                position = -1
                entry_spread = current_spread
                entry_date = z_score.index[i]
        else:
            # Vérifier stop-loss
            stop_hit = False
            
            if stop_type == 'spread_sigma':
                # Stop si spread dépasse threshold * sigma
                current_sigma = spread_std.loc[z_score.index[i]]
                if position == 1 and (current_spread - entry_spread) < -stop_threshold * current_sigma:
                    stop_hit = True
                elif position == -1 and (current_spread - entry_spread) > stop_threshold * current_sigma:
                    stop_hit = True
            
            elif stop_type == 'time_based':
                # Stop si position ouverte > threshold jours
                days_open = (z_score.index[i] - entry_date).days
                if days_open > stop_threshold:
                    stop_hit = True
            
            if stop_hit:
                position = 0
                stopped_out += 1
            # Sortie normale au z-exit
            elif position == 1 and z > z_exit:
                position = 0
            elif position == -1 and z < -z_exit:
                position = 0
        
        signal.iloc[i] = position
    
    # Calculer returns
    spread_returns = spread_series.pct_change().loc[signal.index]
    strategy_returns = spread_returns * signal.shift(1)
    strategy_returns = strategy_returns.dropna()
    
    if len(strategy_returns) > 0 and strategy_returns.std() > 0:
        sharpe = strategy_returns.mean() / strategy_returns.std() * np.sqrt(252)
    else:
        sharpe = 0
    
    return {
        'sharpe': round(sharpe, 3),
        'stopped_out': stopped_out
    }

# Comparer les approches de stop-loss
print("Comparaison des stratégies de stop-loss\n")

stop_configs = [
    ('spread_sigma', 2.5, "Spread-level: 2.5σ"),
    ('spread_sigma', 3.0, "Spread-level: 3.0σ"),
    ('time_based', 60, "Time-based: 60 jours"),
    ('time_based', 90, "Time-based: 90 jours")
]

best_pair = test_pairs.iloc[0]['pair']
etf1, etf2 = best_pair.split('-')

print(f"Test sur la meilleure paire: {best_pair}\n")

for stop_type, threshold, label in stop_configs:
    metrics = pairs_trading_with_stop(
        prices[etf1], prices[etf2],
        stop_type=stop_type,
        stop_threshold=threshold
    )
    
    print(f"{label:30s} → Sharpe={metrics['sharpe']:6.3f}, Stopped={metrics['stopped_out']:3d}")

print("\n⚠ Note: Le stop per-leg actuel (8%) ne peut pas être testé sans données tick")
print("   Recommandation: Utiliser spread-level stop pour préserver la neutralité")

In [ ]:
# Cell 8: Walk-forward validation
def walk_forward_validation(p1, p2, train_days=252, test_days=63):
    """
    Walk-forward validation: train sur N jours, trade sur M jours.
    
    Args:
        train_days: Période d'entraînement (252 = 1 an)
        test_days: Période de test (63 = 3 mois)
    
    Returns:
        list: Résultats par période
    """
    results = []
    
    # Itérer par périodes de test
    for i in range(train_days, len(p1) - test_days, test_days):
        # Fenêtre d'entraînement
        train_p1 = p1.iloc[i-train_days:i]
        train_p2 = p2.iloc[i-train_days:i]
        
        # Fenêtre de test
        test_p1 = p1.iloc[i:i+test_days]
        test_p2 = p2.iloc[i:i+test_days]
        
        # Vérifier la cointégration sur train
        train_coint = test_cointegration(train_p1, train_p2)
        
        if not train_coint['cointegrated']:
            # Skip cette période si pas cointégré
            continue
        
        # Backtest sur la période de test
        test_metrics = pairs_trading_backtest(test_p1, test_p2, z_exit=0.0)
        
        results.append({
            'start_date': test_p1.index[0],
            'end_date': test_p1.index[-1],
            'train_pvalue': train_coint['pvalue'],
            'test_sharpe': test_metrics['sharpe'],
            'test_trades': test_metrics['n_trades']
        })
    
    return results

# Walk-forward sur les 3 meilleures paires
print("Walk-forward validation (train 1 an, test 3 mois)\n")

wf_results_all = []

for _, row in test_pairs.iterrows():
    pair = row['pair']
    etf1, etf2 = pair.split('-')
    
    print(f"\n=== {pair} ===")
    
    wf_results = walk_forward_validation(prices[etf1], prices[etf2])
    
    if len(wf_results) > 0:
        wf_df = pd.DataFrame(wf_results)
        
        # Statistiques
        mean_sharpe = wf_df['test_sharpe'].mean()
        std_sharpe = wf_df['test_sharpe'].std()
        positive_periods = (wf_df['test_sharpe'] > 0).sum()
        
        print(f"Périodes testées: {len(wf_results)}")
        print(f"Sharpe moyen: {mean_sharpe:.3f} ± {std_sharpe:.3f}")
        print(f"Périodes profitables: {positive_periods}/{len(wf_results)} ({100*positive_periods/len(wf_results):.1f}%)")
        
        # Ajouter le nom de paire
        wf_df['pair'] = pair
        wf_results_all.append(wf_df)
    else:
        print("⚠ Aucune période cointégrée trouvée")

# Conclusion sur H5
if len(wf_results_all) > 0:
    all_wf = pd.concat(wf_results_all, ignore_index=True)
    overall_sharpe = all_wf['test_sharpe'].mean()
    
    print(f"\n=== Résultats globaux ===")
    print(f"Sharpe moyen walk-forward: {overall_sharpe:.3f}")
    print(f"Médian: {all_wf['test_sharpe'].median():.3f}")
    print(f"Écart-type: {all_wf['test_sharpe'].std():.3f}")
    
    # On considère que si sharpe WF < 0.3, il y a dégradation significative
    print(f"\n{'✓ H5 CONFIRMÉE' if overall_sharpe < 0.3 else '✗ H5 INFIRMÉE'}: Sharpe WF = {overall_sharpe:.3f} {'<' if overall_sharpe < 0.3 else '>='} 0.3")
else:
    print("\n⚠ Aucun résultat walk-forward disponible")

## Findings - Synthèse des résultats

### Résumé des hypothèses testées

| Hypothèse | Statut | Métrique | Conclusion |
|-----------|--------|----------|------------|
| H1: z-exit 0.5→0.0 améliore Sharpe >0.3 | ... | Δ Sharpe = ... | ... |
| H2: Paires stables <50% du temps | ... | Stabilité moyenne = ...% | ... |
| H3: Half-life > 10 jours | ... | HL médian = ... jours | ... |
| H4: Filtre HL < 30j élimine instables | ... | % paires HL<30j = ...% | ... |
| H5: Dégradation WF >30% | ... | Sharpe WF = ... | ... |

### Corrections prioritaires identifiées

1. **Correction du z-exit** : Passer de 0.5 → 0.0 (attendre la mean-reversion complète)
   - Impact estimé : +... points de Sharpe
   - Justification pédagogique : Le z-score à 0 représente le retour à la moyenne du spread. Sortir à 0.5 revient à quitter la position **avant** que la convergence attendue ne se réalise.

2. **Filtre de half-life** : Exclure les paires avec HL > 30 jours
   - Raison : Half-life trop long → trop de risque de rupture de cointégration pendant la position
   - Pairs restantes : .../... paires (suffisant pour diversification)

3. **Stop-loss spread-level** : Remplacer stop per-leg 8% par spread-level 2.5σ
   - Raison : Le stop per-leg **brise la neutralité** de la position (une jambe peut être stoppée indépendamment)
   - Alternative : Time-based stop à 2× half-life moyen

4. **Durée d'insight adaptative** : Baser sur le half-life au lieu de 6h fixes
   - Formule proposée : `insight_duration = min(2 * half_life, 30 days)`
   - Justification : Le half-life indique la vitesse naturelle de mean-reversion

5. **Extension de période** : 2020-2024 → 2015-2026
   - Objectif : Capturer différents régimes de marché (sideways 2015-2019, volatil 2020-2021)
   - Pré-requis : Appliquer d'abord les corrections 1-4

### Leçons pédagogiques sur le pairs trading

1. **La cointégration n'est pas stationnaire** : Les paires qui sont cointégrées sur le passé peuvent perdre cette propriété.

2. **Le half-life est un signal de robustesse** : Plus il est court, plus la mean-reversion est rapide et fiable.

3. **La neutralité du spread est critique** : Un stop-loss par jambe transforme une position market-neutral en position directionnelle.

4. **Walk-forward validation est essentielle** : Le pairs trading est particulièrement sujet à l'overfitting sur les paramètres de cointégration.

5. **Les coûts de transaction sont cruciaux** : Hourly trading avec trop de rotations (HL < 5 jours) peut tuer la stratégie.

In [ ]:
# Cell 10: Recommandations JSON pour implémentation
import json

# Calculer les métriques finales pour les recommandations
# (Ces valeurs seront remplies après exécution du notebook)

recommendations = {
    "strategy_name": "ETF-Pairs-Trading",
    "current_sharpe": -0.759,
    "target_sharpe": 0.3,
    "corrections": [
        {
            "priority": 1,
            "file": "alpha.py",
            "line": "~130",
            "change": "z_exit: 0.5 → 0.0",
            "code_before": "if abs(z_score) < 0.5:",
            "code_after": "if abs(z_score) < 0.0:",
            "expected_impact": "+0.3 to +0.5 Sharpe points"
        },
        {
            "priority": 2,
            "file": "universe.py",
            "line": "~85",
            "change": "Add half-life filter in pair selection",
            "code_after": "if half_life < 30:  # Filter pairs with HL < 30 days",
            "expected_impact": "+0.1 to +0.2 Sharpe points (stability)"
        },
        {
            "priority": 3,
            "file": "risk.py",
            "line": "~40",
            "change": "Replace per-leg stop with spread-level stop",
            "code_before": "TrailingStopRiskManagementModel(0.08)",
            "code_after": "# Custom spread-level stop at 2.5 sigma",
            "expected_impact": "Preserve market neutrality"
        },
        {
            "priority": 4,
            "file": "alpha.py",
            "line": "~110",
            "change": "Adaptive insight duration based on half-life",
            "code_before": "timedelta(hours=6)",
            "code_after": "timedelta(days=min(2 * half_life, 30))",
            "expected_impact": "Better position timing"
        },
        {
            "priority": 5,
            "file": "main.py",
            "line": "~20",
            "change": "Extend backtest period",
            "code_before": "self.SetStartDate(2020, 1, 1)",
            "code_after": "self.SetStartDate(2015, 1, 1)",
            "expected_impact": "More robust regime testing"
        }
    ],
    "next_steps": [
        "Apply corrections 1-4 to the strategy code",
        "Compile the project on QuantConnect cloud",
        "Run backtest via web UI (2015-2026)",
        "Verify Sharpe > 0.3 before considering live deployment",
        "If still negative, consider daily resolution instead of hourly"
    ],
    "hypothesis_results": {
        "H1_zexit_improvement": "To be filled after execution",
        "H2_stability_pct": "To be filled after execution",
        "H3_median_halflife_days": "To be filled after execution",
        "H4_fast_pairs_pct": "To be filled after execution",
        "H5_wf_sharpe": "To be filled after execution"
    }
}

# Sauvegarder en JSON
output_path = "research_recommendations.json"
with open(output_path, 'w') as f:
    json.dump(recommendations, f, indent=2)

print("✓ Recommandations sauvegardées dans:", output_path)
print("\n" + json.dumps(recommendations, indent=2))